# Train VAE model

This notebook will first try to train the current VAE model before modifying the loss function to work with count data

In [1]:
%load_ext autoreload
%autoreload 2
import os
from ponyo import utils, train_vae_modules

Using TensorFlow backend.


In [2]:
# Set seeds to get reproducible VAE trained models
train_vae_modules.set_all_seeds()

In [3]:
base_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))

# Read in config variables
config_filename = os.path.abspath(
    os.path.join(base_dir, "test_vae_training", "config_current_vae.tsv")
)

params = utils.read_config(config_filename)

dataset_name = params["dataset_name"]

normalized_compendium_filename = params["normalized_compendium_filename"]

In [4]:
# Create VAE directories if needed
output_dirs = [
    os.path.join(base_dir, dataset_name, "models"),
    os.path.join(base_dir, dataset_name, "logs"),
]

NN_architecture = params["NN_architecture"]

# Check if NN architecture directory exist otherwise create
for each_dir in output_dirs:
    sub_dir = os.path.join(each_dir, NN_architecture)
    os.makedirs(sub_dir, exist_ok=True)

In [5]:
# Train VAE on new compendium data
train_vae_modules.train_vae(config_filename, normalized_compendium_filename)

input dataset contains 11857 samples and 1232 genes
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
tracking <tf.Variable 'Variable:0' shape=() dtype=float32> beta
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/home/alexandra/anaconda3/envs/microbe/lib/python3.7/site-packages/keras/engine/training_utils.py:819: UserWarning: Output custom_variational_layer_1 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to custom_variational_layer_1.
  'be expecting any data to be passed to {0}.'.format(name))



Train on 8893 samples, validate on 2964 samples
Epoch 1/40
8893/8893 [==============================] - 63s 7ms/step - loss: -2032937489.9980 - val_loss: -21558715781.8839
Epoch 2/40
8893/8893 [==============================] - 61s 7ms/step - loss: -26884823133.2401 - val_loss: -233159787756.8259
Epoch 3/40
8893/8893 [==============================] - 62s 7ms/step - loss: -99915669208.0729 - val_loss: -287163071262.0567
Epoch 4/40
8893/8893 [==============================] - 62s 7ms/step - loss: -231841317767.2111 - val_loss: -4158022170056.0327
Epoch 5/40
8893/8893 [==============================] - 62s 7ms/step - loss: -434632201390.9079 - val_loss: -1158254590670.5964
Epoch 6/40
8893/8893 [==============================] - 60s 7ms/step - loss: -704234125172.2119 - val_loss: -2658532588407.1904
Epoch 7/40
8893/8893 [==============================] - 57s 6ms/step - loss: -1067440749388.9468 - val_loss: -1713740217450.4075
Epoch 8/40
8893/8893 [==============================] - 57s 6m